# *eBay Kleinanzeigen*: Used Car Analysis

In this project, we're going to use a sampled dataset of 50,000 data points from [this](https://www.kaggle.com/orgesleka/used-cars-database/data) dataset of used cars from *eBay Kleinanzeigen*, a classified section of the German eBay website. 

The data dictionary is as follows:

- dateCrawled - When this ad was first crawled. All field-values are taken from this date.
- name - Name of the car.
- seller - Whether the seller is private or a dealer.
- offerType - The type of listing
- price - The price on the ad to sell the car.
- abtest - Whether the listing is included in an A/B test.
- vehicleType - The vehicle Type.
- yearOfRegistration - The year in which which year the car was first registered.
- gearbox - The transmission type.
- powerPS - The power of the car in PS.
- model - The car model name.
- kilometer - How many kilometers the car has driven.
- monthOfRegistration - The month in which which year the car was first registered.
- fuelType - What type of fuel the car uses.
- brand - The brand of the car.
- notRepairedDamage - If the car has a damage which is not yet repaired.
- dateCreated - The date on which the eBay listing was created.
- nrOfPictures - The number of pictures in the ad.
- postalCode - The postal code for the location of the vehicle.
- lastSeen - When the crawler saw this ad last online

The aim of this project is to clean and analyze this data to provide insight on used car listings.

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')

In [3]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

In [4]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


At a quick glance it looks like there are several columns that demand attention.  `vehicleType`, `gearbox`, `model`, `fuelType`, and `notRepairedDamage` are all short of our 50,000 record count and need to be investigated to see if we should drop the null records or whether we can impute some of the data. The columns `name`, `price` and `odometer` also look to need some cleaning as well before we analyze this dataset.

The first task, however, will be to clean up the column names to be easier to work with.

## Data Cleaning
### Column Names

In [5]:
columns = autos.columns

#Reword some columns to be more descriptive
col_map = {
    "yearOfRegistration":"registration_year",
    "monthOfRegistration":"registration_month",
    "notRepairedDamage":"unrepaired_damage",
    "dateCreated":"ad_created",
    "nrOfPictures":"num_photos"
}

#Break snakecasing
for col in columns:
    if col not in col_map:
        #Regex: insert underscore before capital letters
        #Apply regex and lower, then insert original value and new value into
        #mapping dictionary
        col_map[col] = re.sub(r"(\w)([A-Z])", r"\1_\2", col).lower()
    
autos.rename(col_map, axis='columns', inplace=True)

autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen'],
      dtype='object')

Above the column headers were modified for clarity sake. Snakecase may be okay for programming variables or classes, but when working with data it is a better practice to work with camelcased headers as they are easier to read. 

Some headers were easier to type out to rename since the original wording is unclear, so a mapping dicitonary was initialized with the new names. For the rest a quick regex and lower function was applied and inserted into the mapping dictionary to finish up the camelcase conversion.

### Initial Data Exploration and Cleaning

In [6]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-27 22:55:05,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Looking into the columns there are a few candidates to be dropped:
- `seller` and `offer_type` both offer only 2 unique points and the values look to be mostly the same. These should be considered for dropping because they potentially won't offer much benefit to the analysis
- `num_photos` also looks strange and will need investigating

It can also be noted that `price` and `odometer` are stored as text values and will need some cleaning so they can be converted to numeric values for usage. `price` also has at least one zero value and that will need to be looked into for dropping, as cars are generally not sold for 0.

In [7]:
autos['num_photos'].value_counts()

0    50000
Name: num_photos, dtype: int64

It is quickly obvious that `num_photos` needs to be drop since all values in that column are 0 and won't provide any analysis benefit. We'll drop `seller` and `offer_type` as well.

In [8]:
autos = autos.drop(['num_photos','seller','offer_type'], axis='columns')

As mentioned before, there are a couple of columns that are numeric values with string characters in them that need to be removed. Let's clean and convert those.

In [9]:
autos['price'] = (autos['price']
                 .str.replace("$","")
                 .str.replace(",","")
                 .astype(int)
                )
autos['price'].head()

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int64

In [10]:
autos['odometer'] = (autos['odometer']
                 .str.replace("km","")
                 .str.replace(",","")
                 .astype(int)
                )
autos.rename({"odometer":"odometer_km"}, axis=1, inplace=True)
autos['odometer_km'].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer_km, dtype: int64

Now that those are clean, a quick exploration of the columns to see if there is any strange data that may impact the analysis.

In [11]:
print(autos['price'].shape)
print(autos['price'].describe())
autos['price'].value_counts().sort_index().head(20)

(50000,)
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64

In [12]:
autos['price'].value_counts().sort_index(ascending=False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

The `price` column contains many outliers with values ranging less than 500, and values that look to be fake over 350000. These values are going to skew our results when analyzing prices, so the only rows that will be kept will be between 500 and 350,000. While used cars technically can be sold below 500 and over 350,000, it's not normal, especially on eBay, and there are so few of them compared to the bulk of the data that they will not be missed.

In [13]:
autos = autos[autos['price'].between(500,350000)]
autos['price'].describe()

count     45097.000000
mean       6320.659600
std        9261.841444
min         500.000000
25%        1500.000000
50%        3500.000000
75%        7900.000000
max      350000.000000
Name: price, dtype: float64

In [14]:
print(autos['odometer_km'].shape)
print(autos['odometer_km'].describe())
autos['odometer_km'].value_counts().sort_index().head(20)

(45097,)
count     45097.000000
mean     125293.035013
std       39622.744927
min        5000.000000
25%      100000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64


5000        616
10000       237
20000       727
30000       765
40000       808
50000       996
60000      1131
70000      1189
80000      1385
90000      1676
100000     2031
125000     4838
150000    28698
Name: odometer_km, dtype: int64

It can be seen that the values here in the `odemeter_km` column are rounded, otherwise there would be a lot more unique values for mileage because not every car has exact, round, mileage. Sellers may have been forced to select pre-determined options, which would also explain the "cap" at 150,000; they couldn't select any higher. It should also be noted that there are many more high mileage cars than low mileage.

### Exploring and Cleaning the Date Columns

There are several columns that contain date information:
- date_crawled
- registration_month
- registration_year
- ad_created
- last_seen
These are a combination of dates collected by the crawler and meta-information about the listings. The non-registration dates are stored as strings.

The next goal is to explore these columns to learn more about the listings.

In [15]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


The first ten characters of each column represent the day, if extracted, the distrubution of dates can be seen.

In [16]:
date_crawled = autos['date_crawled'].str[:10]
date_crawled.value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025567
2016-03-06    0.014125
2016-03-07    0.036189
2016-03-08    0.033173
2016-03-09    0.032907
2016-03-10    0.032707
2016-03-11    0.033018
2016-03-12    0.037320
2016-03-13    0.015522
2016-03-14    0.036300
2016-03-15    0.034016
2016-03-16    0.029359
2016-03-17    0.031155
2016-03-18    0.012883
2016-03-19    0.034747
2016-03-20    0.038073
2016-03-21    0.037741
2016-03-22    0.033018
2016-03-23    0.032397
2016-03-24    0.028982
2016-03-25    0.031089
2016-03-26    0.032641
2016-03-27    0.031177
2016-03-28    0.034836
2016-03-29    0.033262
2016-03-30    0.033328
2016-03-31    0.031665
2016-04-01    0.033905
2016-04-02    0.035767
2016-04-03    0.038827
2016-04-04    0.036610
2016-04-05    0.013172
2016-04-06    0.003171
2016-04-07    0.001353
Name: date_crawled, dtype: float64

`date_crawled` contains information that lets us know that the crawler collected information every day for about a month. The distrubution of the dates is mostly consistent ranging from .1% to almost 4% every day, with the dates in the middle containing the most information.

In [17]:
ad_created = autos['ad_created'].str[:10]
ad_created.value_counts(normalize=True, dropna=False).sort_index()

2015-06-11    0.000022
2015-08-10    0.000022
2015-09-09    0.000022
2015-11-10    0.000022
2015-12-05    0.000022
2015-12-30    0.000022
2016-01-03    0.000022
2016-01-07    0.000022
2016-01-10    0.000044
2016-01-13    0.000022
2016-01-14    0.000022
2016-01-16    0.000022
2016-01-22    0.000022
2016-01-27    0.000067
2016-01-29    0.000022
2016-02-01    0.000022
2016-02-02    0.000044
2016-02-05    0.000044
2016-02-07    0.000022
2016-02-08    0.000022
2016-02-09    0.000022
2016-02-11    0.000022
2016-02-12    0.000044
2016-02-14    0.000044
2016-02-16    0.000022
2016-02-17    0.000022
2016-02-18    0.000044
2016-02-19    0.000067
2016-02-20    0.000044
2016-02-21    0.000044
                ...   
2016-03-09    0.032996
2016-03-10    0.032441
2016-03-11    0.033328
2016-03-12    0.037098
2016-03-13    0.016963
2016-03-14    0.034880
2016-03-15    0.033794
2016-03-16    0.029847
2016-03-17    0.030822
2016-03-18    0.013504
2016-03-19    0.033616
2016-03-20    0.038207
2016-03-21 

`ad_created` holds listings that cover almost a year. While they may exist, the older listings are few and far between and as the dates get more recent the proportion of the listings they hold get larger.

In [18]:
last_seen = autos['last_seen'].str[:10]
last_seen.value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.001087
2016-03-06    0.004169
2016-03-07    0.005211
2016-03-08    0.007007
2016-03-09    0.009468
2016-03-10    0.010289
2016-03-11    0.012041
2016-03-12    0.023904
2016-03-13    0.008870
2016-03-14    0.012285
2016-03-15    0.015677
2016-03-16    0.016165
2016-03-17    0.027674
2016-03-18    0.007406
2016-03-19    0.015411
2016-03-20    0.020423
2016-03-21    0.020667
2016-03-22    0.021243
2016-03-23    0.018405
2016-03-24    0.019536
2016-03-25    0.018582
2016-03-26    0.016476
2016-03-27    0.015456
2016-03-28    0.020534
2016-03-29    0.021354
2016-03-30    0.024148
2016-03-31    0.023438
2016-04-01    0.022862
2016-04-02    0.024880
2016-04-03    0.024946
2016-04-04    0.024303
2016-04-05    0.126616
2016-04-06    0.225314
2016-04-07    0.134155
Name: last_seen, dtype: float64

The crawler collected `last_seen`, so it looks that the crawler checked every listing and updated the date it was still open during each crawl. It can be assumed that the auction was closed between the `last_seen` date and the next crawl.

That covers the dates stored as strings, next up, let's grab the distribution of `registration_year` to see what can be learned.

In [19]:
autos['registration_year'].describe()

count    45097.000000
mean      2005.064173
std         89.652017
min       1000.000000
25%       2000.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

It looks like there is some wrong date information in the `registration_year` column. The min value shows that there is at least one row with the year '1000', which is impossible, as well as the max value being '9999' which would mean the car would be from over 7000 years in the future, which is also impossible as this data was collected in 2016.

### Dealing with Incorrect Registration Data

Since the data was collected in 2016, it can be known that any car cannot be registered after 2016. That means 2016 is the upper bound for this analysis, however determining the lower bound is a little trickier. The year 1000 is well before cars were invented, but the realistic year to start is hard to pinpoint. It can be surmised that this date would fall somewhere in the early 1900s.

The first step is to see if it would be safe to remove all cars that fall outside of the 1900-2016 interval. Let's calculate that percentage.

In [20]:
(~autos['registration_year'].between(1900,2016)).sum() / autos.shape[0]

0.039337428210302235

Less than 4% of our data falls outside these bounds, so it is safe to remove these records.

In [21]:
autos = autos[autos['registration_year'].between(1900,2016)]
autos['registration_year'].value_counts(normalize=True).head(10)

2005    0.066108
2000    0.062669
2004    0.061930
2006    0.061538
2003    0.061330
1999    0.059368
2001    0.058168
2002    0.055836
2007    0.052397
2008    0.050920
Name: registration_year, dtype: float64

### Unrepaired Damage
`unrepaired_damage` looks to be an interesting column, however, there are a couple issues that need investigating.
- Translating German words to English
- Deal with missing values

In [22]:
autos['unrepaired_damage'].value_counts(dropna=False)

nein    32992
NaN      6767
ja       3564
Name: unrepaired_damage, dtype: int64

In [23]:
#Translate nein and ja
trans = {
    "nein":"No",
    "ja":"Yes"
}
autos["unrepaired_damage"] = autos["unrepaired_damage"].map(trans)

In [24]:
#Fill NaN with Unknown
autos["unrepaired_damage"].fillna("Unknown", inplace=True)
autos['unrepaired_damage'].value_counts(dropna=False)

No         32992
Unknown     6767
Yes         3564
Name: unrepaired_damage, dtype: int64

Since the missing values comprised of ~18% of our data, dropping the missing values would have severely impacted the rest of the analysis, so for the sake of the rest of the analysis, all NaN values were imputed with Unknown.

# Analysis
Now that the cleaning is done, the analysis can begin.

In this analysis, the following questions will be asked and answered:
- What are the average prices of the top brands?
- What is the relationship between average mileage and average price for those top brands?
- What are the most common brand/model combinations?
- What are the average prices of low, medium and high mileage cars? What is the impact of mileage on price?
- How much cheaper are cars with damage than those without?

## Mean Price
The analysis will start by looking into the unique values of `brand` and make a decision on which values it would be smart to aggregate and analyze the average price.

In [25]:
autos['brand'].unique()

array(['peugeot', 'bmw', 'volkswagen', 'smart', 'ford', 'chrysler',
       'renault', 'audi', 'sonstige_autos', 'mazda', 'porsche', 'mini',
       'mercedes_benz', 'seat', 'toyota', 'opel', 'dacia', 'nissan',
       'jeep', 'saab', 'volvo', 'jaguar', 'fiat', 'skoda', 'subaru',
       'kia', 'citroen', 'mitsubishi', 'chevrolet', 'hyundai', 'honda',
       'daewoo', 'suzuki', 'trabant', 'land_rover', 'alfa_romeo', 'lada',
       'rover', 'daihatsu', 'lancia'], dtype=object)

There are many different brands and doing an analysis on all of them would be technically thorough, not all of the data would be meaningful and could crowd our output. Next we'll look at the proportions of the different brands and make our selection from there.

In [26]:
brand_prop = autos['brand'].value_counts(normalize=True,ascending=False)
print(brand_prop)

volkswagen        0.211897
bmw               0.115020
mercedes_benz     0.102255
opel              0.100339
audi              0.090506
ford              0.065162
renault           0.043903
peugeot           0.029361
fiat              0.023521
seat              0.017889
skoda             0.017150
smart             0.015119
mazda             0.015004
nissan            0.015004
citroen           0.014150
toyota            0.013411
hyundai           0.010156
sonstige_autos    0.009695
volvo             0.009371
mini              0.009371
honda             0.007802
mitsubishi        0.007733
kia               0.007340
alfa_romeo        0.006625
porsche           0.006417
chevrolet         0.006025
suzuki            0.005909
chrysler          0.003601
dacia             0.002839
jeep              0.002447
land_rover        0.002262
daihatsu          0.002170
subaru            0.001985
saab              0.001662
jaguar            0.001616
daewoo            0.001339
rover             0.001246
t

As noted before, there are many brands, but it looks like there are only a few brands that make up a significant portion of the cars listed. Brands like Volkswagen and BMW make up about 20% and 10% respectively while most brands fall way below even 5% of the total listings. 

In this analysis, brands that have a greater than 5% makeup of the listings will be analyzed.

In [27]:
common_brands = brand_prop[brand_prop > .05].index
common_brands

Index(['volkswagen', 'bmw', 'mercedes_benz', 'opel', 'audi', 'ford'], dtype='object')

In [28]:
brand_mean_price = {}

#Collect mean prices
for brand in common_brands:
    mean_price = autos.loc[autos['brand'] == brand,"price"].mean()
    brand_mean_price[brand] = int(mean_price)

#Display mean prices and associated brands
for k, v in brand_mean_price.items():
    print("Brand: {}".format(k))
    print("Avg Price: ${:,}".format(v))
    print("---------------------------")

Brand: bmw
Avg Price: $8,582
---------------------------
Brand: audi
Avg Price: $9,613
---------------------------
Brand: opel
Avg Price: $3,394
---------------------------
Brand: ford
Avg Price: $4,291
---------------------------
Brand: mercedes_benz
Avg Price: $8,766
---------------------------
Brand: volkswagen
Avg Price: $5,783
---------------------------


BMW, Audi, and Mercedes Benz are the most expensive common brands, Ford and Opel are the cheapest, while the most popular car by at least almost double the proportion, Volkswagen, is in the middle. A possible explaination for its popularity is that it has a 'best of both worlds' appeal in both price and luxury. Not as cheap as Ford or Opel, but not nearly as expensive as the other German brands.

Next up, a comparison between mean price and mean mileage will be conducted.
## Mean Price vs. Mean Mileage
The reason to compare these two values is to see if the mileage of these listings impact the price. So let's grab the mean mileage for our common brands and then put them in a dataframe together with the mean price.

In [29]:
brand_mean_mileage = {}

#Collect mean mileage data
for brand in common_brands:
    mean_mileage = autos.loc[autos['brand'] == brand,"odometer_km"].mean()
    brand_mean_mileage[brand] = int(mean_mileage)
    
#Convert both mean dictionaries into Pandas data series
bmp_series = pd.Series(brand_mean_price)
bmm_series = pd.Series(brand_mean_mileage)

mean_comp = pd.DataFrame(bmp_series,columns=['mean_price'])
mean_comp['mean_mileage'] = bmm_series

mean_comp

,mean_price,mean_mileage
audi,9613,128909
bmw,8582,132865
ford,4291,123494
mercedes_benz,8766,131019
opel,3394,128012
volkswagen,5783,128234


The range of mileage does not vary much as does price, since all the mileages fall within 10% of each other. However, this could be skewed since it was noticed earlier that a large proportion (over 50%) of mileage values in the dataset were 150,000 causing the potential variance to be washed away. That being said, the only meaningful analysis that can be made is that there is little to no impact mileage has between the top brands on price. Although, the more expensive brands have a slight trend towards higher mileage than the others.
## Brand/Model Combinations
The best way to analyze this next question is going to be grouping brands and models together and finding the max count for each unique brand and model combination.

In [30]:
#Grab brand/model counts into a dataframe
top_bm = (autos
               .groupby(['brand', 'model'])
               .size()
               .reset_index(name="count")
          )

#Create boolean index of max counts for brand
idx = top_bm.groupby(['brand'])['count'].transform(max) == top_bm['count']

#Apply index and sort
top_bm[idx].sort_values(by='count',ascending=False).head(10)

,brand,model,count
266,volkswagen,golf,3437
23,bmw,3er,2519
181,opel,corsa,1266
13,audi,a4,1199
140,mercedes_benz,c_klasse,1122
81,ford,focus,715
192,peugeot,2_reihe,579
237,smart,fortwo,546
210,renault,twingo,488
73,fiat,punto,321


Unsurprising, based on earlier analysis, Volkswagen comes in on top with the Golf. Following right after are the other common brands from the mean price/mileage analysis. This should be expected due to the proportion difference of listings between the brands, the most common brands have the most common models.

## Mileage Price Average: Low, Medium, and High
To find the average price for each mileage groups, the categories will be as follows:
- Low: Less Than 50k miles
- Medium: 50k - 100k miles
- High: Greater Than 100k miles

After the mean price for each group has been calculated, the groups will be placed into a Pandas Series together to examine.

In [31]:
mileage_prices = {}

mileage_prices['low_miles'] = int((autos.loc[autos['odometer_km']
                                        .between(0,49999),'price']
                                        .mean()
                              ))
mileage_prices['med_miles'] = int((autos.loc[autos['odometer_km']
                                        .between(50000,99999),'price']
                                        .mean()
                              ))
mileage_prices['high_miles'] = int((autos.loc[autos['odometer_km']
                                        .between(100000,150000),'price']
                                        .mean()
                               ))

avg_price_by_mileage = pd.Series(mileage_prices)

avg_price_by_mileage

high_miles     4679
low_miles     16594
med_miles     10980
dtype: int64

Mileage has a large impact on average price with an inverse relationship; the higher the miles, the lower the price. Low mileage price is greater than high mileage by over 3 times, which is a statistically significant difference, therefore it can be safely assumed that mileage is the cause for the price difference.

## Unrepaired Damage Effects On Price
It is common knowledge that cars that are damaged or need repairs done to them are usually cheaper than their undamaged counterparts, since the cost of repairs is reflected often as a cheaper price. However, what is not known is quantifiably HOW much cheaper are the damaged cars. This analysis will attempt to answer that question below.

The steps to be followed:
- Collect the mean price of all damaged and undamaged cars respectively
- Calculate the difference between those two averages

In [32]:
autos['unrepaired_damage'].value_counts()

No         32992
Unknown     6767
Yes         3564
Name: unrepaired_damage, dtype: int64

There are 6767 'Unknown' damage status that will be ignored since the true status cannot be confirmed or inferred from the other columns.

In [33]:
#Filter out Unknown Records
auto_dmg = autos[~(autos['unrepaired_damage'] == 'Unknown')]
values = auto_dmg['unrepaired_damage'].unique()
values

array(['No', 'Yes'], dtype=object)

In [34]:
#Calculate the mean price for Yes and No damage
damaged_prices = {}

for val in values:
    mean = auto_dmg.loc[auto_dmg['unrepaired_damage'] == val, 'price'].mean()
    mean = int(mean)
    damaged_prices[val] = mean

print(damaged_prices)

#Calculate the difference in percentage
diff = (damaged_prices['Yes'] / damaged_prices['No']) * 100
print("Difference: {:.2f}%".format(diff))

{'Yes': 2779, 'No': 7339}
Difference: 37.87%


Vehicles without damage, on average, are significantly priced higher than vehicles with damage. In this case, it can be expected to see close to a 38% decrease in price if a car is damaged. 

However, the challenge with this analysis is how many cars were available to sample from. There were 33,000 cars to sample from that were not damaged compared to around 3,600 cars with damage, with another close to 7,000 lost due to not enough data provided. To properly define the difference in price in these two categories, more data on damaged car listings would need to be collected to comfortably make that analysis since there were nearly 10 times the amount of listings that were not damaged. 